In [ ]:
!pip install plotly-express --quiet


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import plotly


In [ ]:
folder_path = '.'

In [ ]:
precision_recall_filepath = [filename for filename in os.listdir() if filename.startswith('percentwise_precision') and filename.endswith('csv')][0]
clientwise_precision_recall_filepath = [filename for filename in os.listdir() if filename.startswith('percentwise_clientwise_')and filename.endswith('csv')][0]
print('precision_recall_filepath',precision_recall_filepath)
print('clientwise_precision_recall_filepath',clientwise_precision_recall_filepath)

In [ ]:
metadata_df = pd.read_csv('s3://saiva-playground/gill/facility_model_config.csv')
metadata_df.rename(columns={'rank_cutoff': 'cutoff'}, inplace=True)
metadata_df = metadata_df[['client','facilityid','model_golive_date','facility_golive_date','cutoff']]
metadata_df.head()

In [ ]:
precision_recall_filepath

In [ ]:
final_df = pd.read_csv(precision_recall_filepath)
final_df = final_df.merge(metadata_df,how='left',on=['client','facilityid'])
final_df.head()

In [ ]:
# unique client-facility pairs
client_facility_unique_pairs = final_df[['client', 'facilityname']].drop_duplicates()
client_facility_unique_pairs.to_dict('records')

### graph knobs

In [ ]:
client = ['trio','avante','marquis']
# facilityname = [ 'Avante at Ormond Beach']#,'BeaverCreek Health and Rehab']
x_axis = 'recall_at_k%'
y_axis = 'precision_at_k%'
color_group='facilityname'
graph_width = 1200
graph_height = 600
marker_size = 7
cutoff_marker_size = 18
symbol_group='rank'
text = 'info'

## graph plot

In [ ]:
plot_df = final_df[(final_df['client'].isin(client))]

plot_df.loc[:,'info'] = plot_df.loc[:,'rank'].astype(str)+'(rank%)__'
plot_df.loc[:,'info'] += plot_df.loc[:,'rth_predicted'].astype(int).astype(str)+'/'+plot_df.loc[:,'total_residents_transferred'].astype(int).astype(str)+'(r)__'
plot_df.loc[:,'info'] += plot_df.loc[:,'num_precision'].astype(int).astype(str)+'/'+plot_df.loc[:,'denom_precision'].astype(int).astype(str)+'(p)'
plot_df.head()

In [ ]:
title="percentwise precision-recall curve_"+'_to_'.join(precision_recall_filepath.split('_')[-2:]).replace('.csv','')

fig = px.scatter(
    plot_df, 
    x=x_axis, 
    y=y_axis,
    color=color_group,
    title=title,
    width=graph_width, 
    height=graph_height,
    text=text
)
fig.update_traces(marker={'size': marker_size, 'line':{'width':2, 'color':'DarkSlateGrey'}},textfont_size=1)

fig.update_yaxes(rangemode="tozero")
fig.update_xaxes(rangemode="tozero")

fig.show()


In [ ]:
# save dynamic plot as html file
plotly.offline.plot(fig, filename=os.path.join(folder_path,f'{title}.html'))

## clientwise

In [ ]:
clientwise_final_df = pd.read_csv(clientwise_precision_recall_filepath)


In [ ]:
client = ['trio', 'avante', 'marquis']
x_axis = 'recall_at_k%'
y_axis = 'precision_at_k%'
color_group='client'
graph_width = 1200
graph_height = 600
marker_size = 10
# symbol_group='rank_cutoff'

In [ ]:
clientwise_final_df['rank'] = clientwise_final_df['rank'].astype('int') 
clientwise_final_df.sort_values(by=['client','rank'],inplace=True)
clientwise_final_df.head()

In [ ]:
clientwise_plot_df = clientwise_final_df[(clientwise_final_df['client'].isin(client))]
clientwise_plot_df.loc[:,'info'] = clientwise_plot_df.loc[:,'rank'].astype(str)+'(rank%)__'
clientwise_plot_df.loc[:,'info'] += clientwise_plot_df.loc[:,'rth_predicted'].astype(int).astype(str)+'/' + clientwise_plot_df.loc[:,'total_residents_transferred'].astype(int).astype(str)+'(r)__'
clientwise_plot_df.loc[:,'info'] += clientwise_plot_df.loc[:,'num_precision'].astype(int).astype(str)+'/' + clientwise_plot_df.loc[:,'denom_precision'].astype(int).astype(str)+'(p)'


In [ ]:
client_title = "percentwise clientwise precision-recall curve_"+'_to_'.join(clientwise_precision_recall_filepath.split('_')[-2:]).replace('.csv','')
fig = px.scatter(
    clientwise_plot_df, 
    x=x_axis, 
    y=y_axis,
    color=color_group,
    title=client_title,
    width=graph_width, 
    height=graph_height,
    text=text
)
fig.update_traces(marker={'size': marker_size},textfont_size=1)

fig.update_yaxes(rangemode="tozero")
fig.update_xaxes(rangemode="tozero")

fig.show()

In [ ]:
# save dynamic plot as html file
plotly.offline.plot(fig, filename=os.path.join(folder_path,f'{client_title}.html'))